In [ ]:
%load_ext autoreload
%autoreload 2
    
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.optimize import curve_fit

from functionsFIMS import getAnalysisNumbers, plotPolyExamples
from runDataClass import runData
from polyaClass import myPolya
from gasDataClass import magboltzSimulation


In [ ]:
if __name__ == '__main__':
    runNos = getAnalysisNumbers()
    simData = runData(1173)
    treenames = simData.getTreeNames()
    print(treenames)
    for name in treenames:
        simData.printColumns(name)

In [ ]:
if __name__ == '__main__':
    simData.plot2DFieldLines('Cathode')
    simData.plotAllFieldLines()

In [ ]:
test = simData.getDataFrame('metaData')

In [ ]:
print(test)

In [ ]:
plotPolyExamples()

In [ ]:
testPolya = myPolya(gain=60, theta=0.5)
testPolya.solveForGain(targetEff=.92, threshold=10)
print(testPolya.gain)

In [ ]:
print(np.arange(0, 3.1, 1))

In [ ]:
def requiredGain(theta, efficiency=.95, threshold=10):
    """
    TODO
    """
    from scipy.special import gammaincinv
    
    if not (0 < efficiency < 1):
        raise ValueError('Efficiency must be between 0 and 1.')
    if threshold <= 0:
        raise ValueError('Threshold must be positive.')

    alpha = theta + 1
    pLower = 1.0 - efficiency

    try:
        xArg = gammaincinv(alpha, pLower)
    except Exception as e:
        print(f"Error during gammaincinv calculation: {e}")
        raise

    nominalGain = (threshold * alpha) / xArg

    gainResult = np.where(xArg <= 0, np.inf, nominalGain)
    
    return gainResult

from scipy.optimize import curve_fit
def exponentialDecay(x, a, b, c):
    return a*np.exp(-b*x) + c

def fitExponentialDecay(thetaValues, gainValues):

    from scipy.stats import chi2 as chi2Dist

    popt, pcov = curve_fit(exponentialDecay, thetaValues, gainValues, p0=[100, 0.1, 1])
    perr = np.sqrt(np.diag(pcov))

    yFit = exponentialDecay(thetaValues, *popt)
    residuals = gainValues - yFit
    dof = len(thetaValues)-len(popt)
    chi2 = np.sum(residuals**2)
    rChi2 = chi2/dof
    pVal = chi2Dist.sf(chi2, dof)
    
    chi2Info = {
        'chi2': chi2, 
        'dof': dof,
        'rChi2': rChi2,
        'pVal': pVal
    }

    return popt, perr, chi2Info


def plotRequiredGains():
    """
    TODO
    """

    theta = np.linspace(0, 2, 31)
    fitTheta = np.linspace(0,2,100)

    plotConfigs = [
        {
            'eff': 0.95, 'thresh': 10, 
            'c': 'b', 'marker': 'o'
        },
        {
            'eff': 0.95, 'thresh': 12, 
            'c': 'g', 'marker': 'x'
        },
        {
            'eff': 0.95, 'thresh': 8, 
            'c': 'r', 'marker': 'x'
        },
        {
            'eff': 0.94, 'thresh': 10, 
            'c': 'm', 'marker': '+'
        },
        {
            'eff': 0.96, 'thresh': 10, 
            'c': 'c', 'marker': '+'
        }
    ]

    nominalVals = plotConfigs[0]
    gainsNominal = requiredGain(theta, efficiency=nominalVals['eff'], threshold=nominalVals['thresh'])
    fitParam, fitParamErr, chi2Info = fitExponentialDecay(theta, gainsNominal) 
    

    fig = plt.figure(figsize=(12, 15))
    fig.suptitle('Gain Sensitivity')
    ax = fig.add_subplot(311)
    axNorm = fig.add_subplot(312)
    axNominal = fig.add_subplot(313)

    for inConfig in plotConfigs:
        
        inGain = requiredGain(theta, efficiency=inConfig['eff'], threshold=inConfig['thresh'])
        ax.scatter(
            theta, inGain,
            c=inConfig['c'], marker=inConfig['marker'],
            label=f'Efficiency = {inConfig['eff']}, Threshold = {inConfig['thresh']}'
        )
        axNorm.plot(
            theta, inGain/gainsNominal,
            c=inConfig['c'], marker=inConfig['marker'],
            label=f'Efficiency = {inConfig['eff']}, Threshold = {inConfig['thresh']}'
        )   

    fitResults = exponentialDecay(fitTheta, *fitParam)
    ax.plot(
            fitTheta, fitResults,
            c='b', ls='--',
            label=f'Exponential Fit (rChi2 = {chi2Info['rChi2']:.2f}, pVal = {chi2Info['pVal']:.1e})'
        )
    gainTheta0 = requiredGain(0, efficiency=.95, threshold=10)
    axNominal.scatter(
            theta, gainsNominal/gainTheta0,
            c='b', marker='o',
            label=f'Efficiency = {nominalVals['eff']}, Threshold = {nominalVals['thresh']}\nNormalized to gain at Theta = 0'
        )
    gainFitTheta0 = exponentialDecay(0, *fitParam)
    axNominal.plot(
            fitTheta, fitResults/gainFitTheta0,
            c='b', ls='--',
            label=f'Normalized Exponential Fit (rChi2 = {chi2Info['rChi2']:.2f}, pVal = {chi2Info['pVal']:.1e})'
        )


    ax.set_title('Gain vs. Polya Shape')
    ax.set_xlabel('Polya Shape (Theta)')
    ax.set_ylabel('Gas Gain')
    ax.grid()
    ax.legend()

    axNorm.set_title('Normalized Gain vs. Polya Shape')
    axNorm.set_xlabel('Polya Shape (Theta)')
    axNorm.set_ylabel('Normalized Gas Gain')
    axNorm.grid()
    axNorm.legend()

    axNominal.set_title('Normalized Gain vs. Polya Shape')
    axNominal.set_xlabel('Polya Shape (Theta)')
    axNominal.set_ylabel('Normalized Gas Gain')
    axNominal.grid()
    axNominal.legend()
    
    return

plotRequiredGains()

In [ ]:
def myBayesian(efficiency, numTotal, numSuccess):
    scale = math.factorial(numTotal+1)/(math.factorial(numSuccess)*math.factorial(numTotal-numSuccess))
    probDensity = scale * efficiency**numSuccess * (1-efficiency)**(numTotal-numSuccess)
    return probDensity/numTotal

def myBinomial(efficiency, numTotal, numSuccess):
    scale = math.factorial(numTotal)/(math.factorial(numSuccess)*math.factorial(numTotal-numSuccess))
    probDensity = scale * efficiency**numSuccess * (1-efficiency)**(numTotal-numSuccess)
    return probDensity
    

def plotBayesian(numTotal, numSuccess):
    """
    """
    stepSize = 0.001
    
    mode = numSuccess/numTotal
    mean = (numSuccess+1)/(numTotal+2)
    var = mean*(numSuccess+2)/(numTotal+3) - mean**2
    
    eff = np.arange(0, 1+stepSize, stepSize)
    bayesianProb = myBayesian(eff, numTotal, numSuccess)
    sumBayesian = np.sum(bayesianProb)*stepSize

    binomialProb = myBinomial(eff, numTotal, numSuccess)
    sumBinomial = np.sum(binomialProb)*stepSize

    fig = plt.figure(figsize=(5, 5))
    fig.suptitle('Bayesian Statistics')
    ax = fig.add_subplot(111)

    ax.plot(
        eff, binomialProb,
        c='g', label='PDF - Binomial'
    )
    
    ax.plot(
        eff, bayesianProb,
        c='r', label='PDF - Bayesian'
    )
    ax.plot(
        [mean, mean], 
        [0, myBayesian(mean, numTotal, numSuccess)],
        c='r', ls='--', label=f'Mean={mean:.2f}'
    )
    ax.plot(
        [mode, mode],
        [0, myBayesian(mode, numTotal, numSuccess)],
        c='r', ls=':', label=f'Mode={mode:.2f}'
    )

    sigma1 = np.arange(mean-math.sqrt(var), mean+math.sqrt(var), stepSize)
    sigma1 = np.minimum(sigma1, 1)
    sigma1Prob = myBayesian(sigma1, numTotal, numSuccess)
    sigma1Sum = np.sum(sigma1Prob)*stepSize

    ax.fill_between(
        sigma1, sigma1Prob,
        facecolor='r', label=f'1-Sigma={math.sqrt(var):.2f}\nC.L.={sigma1Sum/sumBayesian:.3f}', alpha=.3
    )

    sigma2 = np.arange(mean-2*math.sqrt(var), mean+2*math.sqrt(var), stepSize)
    sigma2 = np.minimum(sigma2, 1)
    sigma2Prob = myBayesian(sigma2, numTotal, numSuccess)
    sigma2Sum = np.sum(sigma2Prob)*stepSize
    ax.fill_between(
        sigma2, sigma2Prob,
        facecolor='r', label=f'2-Sigma={2*math.sqrt(var):.2f}\nC.L.={sigma2Sum/sumBayesian:.3f}', alpha=.2
    )

    ax.set_title(f'Total (N)={numTotal}, Successes (m)={numSuccess}')
    ax.set_xlabel('Efficiency')
    ax.set_ylabel('P( eff; m, N )')
    ax.grid()
    ax.legend()

plotBayesian(30, 25)

In [ ]:
pureAr = magboltzSimulation('myT2K-100-0-0')
pureAr.setPlotParams('Pure Ar', 'b')

onlyCF4 = magboltzSimulation('myT2K-97-3-0')
onlyCF4.setPlotParams('Ar + 3% CF4', 'g')
onlyIsobutane = magboltzSimulation('myT2K-98-0-2')
onlyIsobutane.setPlotParams('Ar + 2% Isobutane', 'r')

baseT2K = magboltzSimulation('myT2K-95-3-2')
baseT2K.setPlotParams('Base T2K', 'b')

moreCF4 = magboltzSimulation('myT2K-93-5-2')
moreCF4.setPlotParams('2% More CF4', 'g')
lessCF4 = magboltzSimulation('myT2K-97-1-2')
lessCF4.setPlotParams('2% Less CF4', 'r')

moreIsobutane = magboltzSimulation('myT2K-93-3-4')
moreIsobutane.setPlotParams('2% More Isobutane', 'g')
lessIsobutane = magboltzSimulation('myT2K-96-3-1')
lessIsobutane.setPlotParams('1% Less Isobutane', 'r')

In [ ]:
fig, axes = pureAr.plotMagboltz()
onlyCF4.plotMagboltz(fig, axes)
onlyIsobutane.plotMagboltz(fig, axes)
plt.show()

In [ ]:
fig, axes = baseT2K.plotMagboltz()
moreCF4.plotMagboltz(fig, axes)
lessCF4.plotMagboltz(fig, axes)
plt.show()

In [ ]:
fig, axes = baseT2K.plotMagboltz()
moreIsobutane.plotMagboltz(fig, axes)
lessIsobutane.plotMagboltz(fig, axes)
plt.show()

In [ ]:
TEST = magboltzSimulation()
TEST.loadOptimalDrift('myT2K')
_ = TEST.plotOptimalParam()